University of Helsinki, Master's Programme in Mathematics and Statistics  
MAST32001 Computational Statistics, Autumn 2022  
Luigi Acerbi  

# Week 5 exercises

For this week exercises, you can implement your code using either Autograd or PyTorch as you prefer.

Autograd provides a simpler, more NumPy-like interface, while PyTorch provides a more powerful framework for advanced users. If in doubt, it is probably better to try Autograd. You can port your code to PyTorch later if you wish to experiment with that.

Please see Chapter 10 of the course notes and this week's computer tasks for more details and examples of use of both frameworks.

-----------------------------------

*Hints*:

- If you are using Autograd, make sure you always import `autograd.numpy` instead of regular `numpy` and `autograd.scipy` instead of regular `scipy` to allow Autograd to differentiate any functions you use. This can usually be achieved easily using
  ```
  import autograd.numpy as np
  import autograd.scipy.special as scs
  ```
  and using the packages via `np` and `scs` aliases as usual.
- The import trick above is not needed for PyTorch, which does not support differentiation of NumPy and SciPy functions, only its own functions.
- If you are having problems with Autograd, please note the following:
  * Autograd does not support subscripted assignment (e.g. `a[1] = 2`) and will give a mysterious error if you try.
  * Autograd does not support named function arguments (e.g. `f(foo=1, bar=2)`) and will give a mysterious error if you try.
  * Autograd can only differentiate functions in `autograd.numpy` and `autograd.scipy`. Check your imports if you get mysterious errors.
- If you use PyTorch, please make sure to use the `double` data type. The accuracy of the default data type `float` may not be sufficient.

## 1. Gradients of a multivariate normal likelihood (4 points)

We will test the evaluation and gradient of the multivariate normal likelihood using a covariance matrix specified by a Gaussian process. A 1D Gaussian process is a distribution over continuous functions such that the joint distribution over any finite set of points is multivariate normal (Gaussian). We will use the so-called *squared exponential* covariance function over input points $t_1, \dots, t_n$ defined by  
$$ k(t_i, t_j; \ell) = \exp\left( -\frac{(t_i-t_j)^2}{2 \ell^2} \right), $$
where the parameter $\ell$ is called the *length scale* of the process.

1. Write a function to generate a $3 \times 3$ covariance matrix $K$ such that $K_{i,j} = k(t_i, t_j; \ell)$ where $t_1 = 1, t_2 = 2, t_3 = 3$ as a function of $\ell$.
2. Evaluate the log-pdf of the vector $x = (1, 0, 1)$ under a multivariate normal with $\textbf{0}$ mean and the above covariance matrix $K$ with $\ell = 2$. Report the value of the log-pdf to Moodle.
3. Evaluate the finite difference approximation to the gradient of the log-pdf with respect to $\ell$ at the point $\ell=2, x=(1, 0, 1)$ with $h = 10^{-3}$ (`1e-3`) and report it to Moodle. *Note*: Use the first-order finite difference formula presented in Section 10.2 of the lecture notes
4. Evaluate the exact gradient of the log-pdf with respect to $\ell$ at the point $\ell = 2, x = (1, 0, 1)$ using automatic differentiation and report it to Moodle.

*Hint*: you can generate a matrix of differences of $t$ values in NumPy using the `None` index to create a new dimension:
```
t = np.array([1.0, 2.0, 3.0])
dt = t[:,None] - t[None,:]
```

In PyTorch, the corresponding trick works but the input needs to be a `tensor`:
```
torch.set_default_dtype(torch.double)
# Uncomment this to run on GPU:
# torch.set_default_tensor_type(torch.cuda.DoubleTensor)

t = torch.tensor([1.0, 2.0, 3.0])
dt = t[:,None] - t[None,:]
```

Useful PyTorch functions for evaluating the multivariate normal density that differ from NumPy: `torch.linalg.solve()`, `torch.dot()`

In [1]:
import torch
import numpy as np

torch.set_default_dtype(torch.double)
# Uncomment this to run on GPU:
torch.set_default_tensor_type(torch.cuda.DoubleTensor)

# Part 1
t = torch.tensor([1.0, 2.0, 3.0])
dt = t[:,None] - t[None,:]

def generateCov(l, n = 3):
    cov_matrix = torch.empty( (n, n) )
    for i in range(n):
        for j in range(n):
            cov_matrix[i][j] = torch.exp( -(t[i]-t[j])**2/ (2*l**2) )
    return cov_matrix

sigma = generateCov(2)

print(sigma)

tensor([[1.0000, 0.8825, 0.6065],
        [0.8825, 1.0000, 0.8825],
        [0.6065, 0.8825, 1.0000]])


In [2]:
# Part 2

def logDetSigma(sigma):
    diag = sigma.diagonal()
    diag = torch.log(diag)
    return 2*torch.sum(diag)

def quadFormEval(L, x, mu):
    z = torch.linalg.solve_triangular(L, x-mu, upper=False)
    return torch.t(z) @ z

def calcLnP(x, mu = torch.zeros( (1,3) ).t(), sigma = sigma):
    L = torch.linalg.cholesky(sigma, upper=False)
    d = sigma.shape[0]
    return -0.5*d*torch.log(2*torch.Tensor([np.pi])) - 0.5*logDetSigma(L) - 0.5*quadFormEval(L, x, mu)

x = torch.Tensor( [[1,0,1]] ).t()

mu = torch.zeros( (1,3) ).t()

l = torch.tensor( 2.0, requires_grad=True )

log_pdf = calcLnP( x, mu, generateCov(l) )
print('The log PDF is given by: ', log_pdf.item())

The log PDF is given by:  -21.219486088440906


In [3]:
# Part 3

print( (calcLnP(x, sigma=generateCov(2+1e-3)) - calcLnP(x, sigma=generateCov(2)))/1e-3 )

tensor([[-34.7384]])


In [4]:
# Part 4

l = torch.tensor( 2.0, requires_grad=True )

v = calcLnP(x, sigma=generateCov(l))
v.backward()
print(l.grad.item())

-34.71313691474738


## 2. Maximum likelihood estimation of the gamma distribution (4 points)

In this exercise we test maximum likelihood estimation of the [Gamma distribution](https://en.wikipedia.org/wiki/Gamma_distribution) to a given data set.

Note that the parameters $\alpha$ and $\beta$ of the Gamma distribution need to be positive. This can be accomplised by using $a$ and $b$ satisfying $\alpha = \exp(a)$ and $\beta = \exp(b)$ as variables in the optimization.

1. Load the data set $\mathcal{D} = \{ x_i | i = 1, \dots, n \}$ using the function below.
2. Find the maximum likelihood fit for the parameters $\alpha$ and $\beta$ of the Gamma distribution $\Gamma(\alpha, \beta)$ when $x_i \sim \Gamma(\alpha, \beta)$.
3. Plot the maximum likelihood distribution together with a normed histogram of the data to check your estimates.
4. Report your estimates $\alpha$ and $\beta$ in Moodle. (*Note*: please remember to report $\alpha$ and $\beta$, not $a$ and $b$.)

*Hints*: 
- Feel free to use the implementation of the Gamma distribution pdf provided below. Note that here we are using the shape-rate parametrization of the Gamma pdf.
- Note that we are performing maximum-likelihood estimation, not Bayesian inference. There is no prior here. So, there is no prior density that needs to be corrected when you transform from $\alpha \rightarrow a$ and $\beta \rightarrow b$.

In [14]:
# Autograd code
import autograd.scipy.special as scs
import autograd.numpy as np
import pandas as pd
from scipy.optimize import minimize
import autograd

data = pd.read_csv('https://www2.helsinki.fi/sites/default/files/atoms/files/toydata.txt', sep='\t', header=None).values

def gamma_logpdf_autograd(x, alpha, beta):
    """Log pdf of the Gamma distribution."""
    return (alpha*np.log(beta) - scs.gammaln(alpha) + (alpha-1) * np.log(x) - beta * x)


In [6]:
# PyTorch code
%matplotlib inline
import torch
import pandas as pd
import matplotlib.pyplot as plt

data = torch.tensor(pd.read_csv('https://www2.helsinki.fi/sites/default/files/atoms/files/toydata.txt', sep='\t', header=None).values, dtype=torch.double)

def gamma_logpdf_pytorch(x, alpha, beta):
    """Log pdf of the Gamma distribution."""    
    return (alpha*torch.log(beta) - torch.lgamma(alpha) + (alpha-1) * torch.log(x) - beta * x)

x = torch.tensor([1.0, 2.0], requires_grad=True)

maxiters = 1000
x_history = np.nan*np.zeros((maxiters,2))

def closure():
    optimizer.zero_grad()
    l = -torch.sum(gamma_logpdf_pytorch(data, torch.exp(x[0]), torch.exp(x[1])))
    l.backward()
    return l

optimizer = torch.optim.LBFGS([x],max_iter=maxiters)
optimizer.step(closure)

print("The value of alpha is: ", torch.exp(x[0]).item() )
print("The value of beta is: ", torch.exp(x[1]).item() )

The value of alpha is:  6.495221194297738
The value of beta is:  1.6332627237281399


## 3. Student-$t$ regression (6 points)

In this task we will apply linear regression to data from the Framingham Heart Study that studies the association between heart disease and its causes.

A description of the data and its fields can be found at http://www.maths.utas.edu.au/DHStat/Data/Flow.html (see also problem 10.4 in the computer tasks done in class).

In this exercise, we will use the [Student-$t$ distribution](https://en.wikipedia.org/wiki/Student%27s_t-distribution) for the noise. This provides a form of robust regression due to the heavy tails of the Student-$t$ distribution, which makes it less affected by outliers.

Linear regression fit to scalar $x_i, y_i$ involves fitting the model
$$ y_i = \alpha + \beta x_i + \epsilon_i, $$
where $\beta$ is the regression coefficient and $\alpha$ is the intercept. Assuming $\epsilon_i \sim p(\epsilon_i | \sigma)$, the log-likelihood of the model is
$$ \log p(Y | X, \alpha, \beta, \sigma) = \sum_{i=1}^n \log p(y_i | x_i, \alpha, \beta, \sigma)
  = \sum_{i=1}^n \log p_{\epsilon_i}(y_i - \alpha - \beta x_i;\; \sigma), $$
where $p_{\epsilon_i}(y_i - \alpha - \beta x_i;\; \sigma)$ is the probability density of
the noise $\epsilon_i$, evaluated at $y_i - \alpha - \beta x_i$. 
In Problem 3 from Week 2, and in the computer task 10.3 and 10.4, we used a normal distribution for $p_{\epsilon_i}$ whereas here we will use the Student-$t$ distribution with $\nu = 5$ degrees of freedom and scale parameter $\sigma = 1$.

1. Load the data using the below code and plot the data.
2. Fit the data using standard linear regression with normally distributed errors ($p_{\epsilon_i} = \mathcal{N}(\epsilon_i;\; 0, \sigma^2)$ with $\sigma^2 = 1$). Report the $\alpha$ and $\beta$ you found in Moodle. (*Hint*: Problem 3 from Week 2 contains the formulas for $\alpha$ and $\beta$.)
3. Implement linear regression using the Student-$t$ log-likelihood function ($p_{\epsilon_i} = \text{Student-}t(\epsilon_i;\; 0, \sigma, \nu)$) with degrees of freedom (df) $\nu = 5$ and $\sigma = 1$, and maximise it (i.e. minimise the negative log-likelihood). Plot the fitted regression line in the same figure as the data.
4. Compare the coefficients $\alpha$ and $\beta$ you found for Student-$t$ regression with those from the standard linear regression. What can you observe? Report the coefficients you found in Moodle.

*Hint*: Feel free to use the Student-$t$ log-likelihood function implementations provided below.

In [7]:
# Code for autograd
%matplotlib inline
import pandas as pd
import autograd.numpy as np
import autograd.scipy.special as scs
import autograd
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# load the data from CSV file using pandas
fram = pd.read_csv('https://www2.helsinki.fi/sites/default/files/atoms/files/fram.txt', sep='\t')
# convert the variables of interest to numpy arrays
# input: Blood cholesterol level
x = fram['CHOL'].values
# target: Systolic blood pressure, examination 1
y = fram['SBP'].values

def student_logpdf_autograd(x, nu):
    """Log pdf of Student-t distribution with nu degrees of freedom."""
    return scs.gammaln(0.5*(nu+1)) - scs.gammaln(0.5*nu) - 0.5*np.log(nu*np.pi) - 0.5*(nu+1)*np.log(1 + x**2/nu)

In [10]:
# Code for PyTorch
%matplotlib inline
import pandas as pd
import torch
import math
import matplotlib.pyplot as plt

# load the data from CSV file using pandas
fram = pd.read_csv('https://www2.helsinki.fi/sites/default/files/atoms/files/fram.txt', sep='\t')
# convert the variables of interest to numpy arrays
# input: Framingham relative weight - the ratio of the subjects weight to the median weight for their sex-height group
x = torch.tensor(fram['CHOL'].values, dtype=torch.double)
# target: Systolic blood pressure, examination 1
y = torch.tensor(fram['SBP'].values, dtype=torch.double)

def student_logpdf_pytorch(x, nu):
    """Log pdf of Student-t distribution with nu degrees of freedom."""    
    return torch.lgamma(0.5*(nu+1)) - torch.lgamma(0.5*nu) - 0.5*math.log(math.pi) - 0.5*torch.log(nu) - 0.5*(nu+1)*torch.log(1 + x**2/nu)


# Part 1
def estimator(x, y):
    n = len(x)
    x_mean = torch.mean(x)
    y_mean = torch.mean(y)
    num = 0
    den = 0
    for i in range(n):
        num += (x[i] - x_mean)*(y[i] - y_mean)
        den += (x[i] - x_mean)**2
    beta = num / den
    alpha = y_mean - beta*x_mean
    return alpha, beta

print(estimator(x, y))

(tensor(131.7322), tensor(0.0697))


In [21]:
# Part 2

params = torch.tensor([131.0, 0.05], requires_grad=True)

maxiters = 1000

def closure():
    optimizer.zero_grad()
    l = -torch.sum(student_logpdf_pytorch(y - params[0] - params[1]*x, torch.tensor(5.0)))
    l.backward()
    return l

optimizer = torch.optim.LBFGS([params], lr=1e-2, max_iter=maxiters)
optimizer.step(closure)

print("The value of alpha is: ", params[0].item() )
print("The value of beta is: ", params[1].item() )

The value of alpha is:  137.91229035123644
The value of beta is:  0.008296125311382633


## 4. Variational inference for a mixture model (6 points)

In this problem, you are asked to estimate the posterior over the parameters of a mixture model using variational inference.

Find the posterior for the mixing weight $w$ and means $\mu_1, \mu_2$ in the normal mixture model
$$ p(x_i | \pi, \mu_1, \mu_2) = w \mathcal{N}(x_i;\; \mu_1, 1^2) + (1-w) \mathcal{N}(x_i;\; \mu_2, 1^2) $$
for the data set loaded below.

In order to ensure $0 \le w \le 1$, we will reparametrise it using the logistic transformation as $w = 1 / (1 + \exp(-\gamma))$. 

We will use normal priors for $\mu_1, \mu_2$ and a logistic-normal prior for $w$, which is equivalent to setting a normal prior for $\gamma$. The exact priors are
$$ p(\mu_1) = \mathcal{N}(\mu_1;\; 0, \sqrt{10}^2) \quad 
   p(\mu_2) = \mathcal{N}(\mu_2;\; 0, \sqrt{10}^2) \quad
   p(\gamma) = \mathcal{N}(\gamma;\; 0, 1.78^2), $$
and as usual we assume factorized (i.e., independent) priors, $p(\mu_1, \mu_2, \gamma) = p(\mu_1) p(\mu_2) p(\gamma)$.

As variational distribution we will use a product of independent distributions,

$$q(\mu_1, \mu_2, \gamma) = q(\mu_1) \cdot q(\mu_2) \cdot q(\gamma)$$

where  $q(\mu_1)$, $q(\mu_2)$ and $q(\gamma)$ are normal distributions. Fit the variational approximation $q(\mu_1, \mu_2, \gamma)$ to the posterior distribution of the model.

Use the stochastic gradient algorithm introduced in Chapter 12 of the course notes (doubly-stochastic variational inference,  DSVI). Please note that the independence assumption above is equivalent to saying that $q(\mu_1, \mu_2, \gamma)$ is a multivariate normal with diagonal covariance matrix $\Sigma$.

Report the means and standard deviations of $q(\mu_1)$ and $q(\mu_2)$, and the mean and standard deviation of $w$ under $q(\gamma)$ in Moodle. The required tolerance is $\pm 0.03$. When reporting the values to Moodle, please order the components so that the mean of the first component is lower than the mean of the second component, i.e. $\mathbb{E}[\mu_1] < \mathbb{E}[\mu_2]$.

*Hints*: 
- Computing the mean and standard deviation of $w$ requires extra effort because the approximation $q(\gamma)$ is over $\gamma$. The easiest way to solve this problem is to draw a number of samples from $q(\gamma)$, transform those to $w$ and compute the mean and standard deviation of the transformed samples.
- Note that the prior $p(\gamma)$ is already defined in the transformed (unconstrained) space, so you do not need to apply the Jacobian correction to the density used when transforming variables.
- You can reuse the code for DSVI provided in the class computer tasks, but note that it works for 1-D targets. You will need to adapt it to work with a 3-D target (or otherwise write your own code from scratch).
- Be sure that you report positive values for the standard deviation.

In [9]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

dataframe = pd.read_csv('https://www2.helsinki.fi/sites/default/files/atoms/files/mixture_data2.txt', header=None, sep='\t')
mixdata = dataframe.values[:,0]
print(mixdata.mean(), mixdata.std())

0.6174781875 1.526910779221377
